In [2]:
import pandas as pd
import numpy as np
import requests
import regex as re
import geopandas, shapely
import json
from functools import reduce
import operator
from pymongo import MongoClient,GEOSPHERE

In [14]:
# Set connection with MongoDB
conn = MongoClient("localhost:27017")
db = conn.get_database("final_project")
p = db.get_collection("places")
h = db.get_collection("homes")

In [15]:
p.create_index([("geometry",GEOSPHERE)])
h.create_index([("geometry",GEOSPHERE)])

'geometry_2dsphere'

In [16]:
def insert_object(df, collection):
    '''
    This function inserts all the information from the dataframe we created with the previous function as a Mongodb object.
    This object will have two field: title (name of the film or show) and reviews (array with all the reviews users have made)
    '''

    collection.insert_many(df.to_dict('records'))

    return "Data succesfully added"

In [520]:
# Download JSON file 
data = json.load(open("data\\data_vivienda.json", 'r', encoding='utf-8'))

In [521]:
data_vivienda = pd.DataFrame(data)
data_vivienda.columns

Index(['propertyCode', 'thumbnail', 'externalReference', 'numPhotos', 'floor',
       'price', 'propertyType', 'operation', 'size', 'exterior', 'rooms',
       'bathrooms', 'address', 'province', 'municipality', 'district',
       'country', 'neighborhood', 'latitude', 'longitude', 'showAddress',
       'url', 'hasVideo', 'status', 'newDevelopment', 'hasLift', 'priceByArea',
       'detailedType', 'suggestedTexts', 'hasPlan', 'has3DTour', 'has360',
       'parkingSpace', 'newDevelopmentFinished'],
      dtype='object')

In [522]:
data_vivienda = data_vivienda.drop(["thumbnail","propertyCode", "externalReference", "numPhotos", "hasVideo", "priceByArea", "detailedType", "country", "suggestedTexts", "hasPlan", "has3DTour", "has360", "newDevelopmentFinished"], axis = 1)

In [523]:
# I'm only interested in those houses inside Madrid, so I'm going to delete all observations that belong to another municipality
data_vivienda = data_vivienda[data_vivienda["municipality"]=="Madrid"]

In [8]:
data_vivienda.dtypes

floor              object
price             float64
propertyType       object
operation          object
size              float64
exterior             bool
rooms               int64
bathrooms           int64
address            object
province           object
municipality       object
district           object
neighborhood       object
latitude          float64
longitude         float64
showAddress          bool
url                object
status             object
newDevelopment       bool
hasLift            object
parkingSpace       object
dtype: object

In [9]:
data_vivienda.isnull().sum()
# Parkingspace has too many null values, and as we cannot fill the with a proper value the best option is to drop the whole column (explain in API the reason for the lack of information)
# Let's check the floor, status and hasLift variables

floor              108
price                0
propertyType         0
operation            0
size                 0
exterior             0
rooms                0
bathrooms            0
address              0
province             0
municipality         0
district             0
neighborhood         0
latitude             0
longitude            0
showAddress          0
url                  0
status             556
newDevelopment       0
hasLift            154
parkingSpace      6018
dtype: int64

In [524]:
data_vivienda = data_vivienda.drop("parkingSpace", axis = 1)

In [480]:
data_vivienda.status.value_counts()
# In this case we are going to fill the null values with 'unspecified'

good              8000
renew               24
newdevelopment       8
Name: status, dtype: int64

In [525]:
# There a couple of values that are not numbers, so I'm going to change them to a value that can be valid. In this case '0', as after looking a Idelista we can assume they are street level homes
data_vivienda.floor = data_vivienda.floor.replace(["bj","en","st","ss"], "0", regex = True)

In [526]:
data_vivienda.status = data_vivienda.status.fillna('unspecified')

In [527]:
data_vivienda.hasLift = data_vivienda.hasLift.fillna("unspecified")

In [528]:
data_vivienda = data_vivienda.dropna(how = 'any')

In [17]:
def geo_frame(df):
    df = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.latitude, df.longitude))
    df['geometry']=df['geometry'].apply(lambda x:shapely.geometry.mapping(x))
    return df

In [529]:
data_vivienda = geo_frame(data_vivienda)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [530]:
# As I have limited resources, and the number of queries I can do in all APIs is limited, I'm going to focus on three district with the more number of houses available

data_vivienda = data_vivienda[data_vivienda.groupby('district').district.transform('count')>71]

In [17]:
test.district.value_counts()

Centro       1741
Salamanca    1727
Chamberí      972
Chamartín     813
Name: district, dtype: int64

In [513]:
data_vivienda.rooms.value_counts()

2    2709
1    2083
3    1822
4     833
5     270
Name: rooms, dtype: int64

In [531]:
data_vivienda = data_vivienda[data_vivienda.groupby('rooms').rooms.transform('count')>70]

In [532]:
data_vivienda = data_vivienda.drop(data_vivienda[data_vivienda.rooms == 0].index)

In [533]:
data_vivienda = data_vivienda[data_vivienda.groupby('floor').floor.transform('count')>20]

In [ ]:
data_test = data[data.price < 7000]

In [537]:
insert_object(data_vivienda, h)

'Data succesfully added'

In [494]:
districts = [i for i in data_vivienda.district.unique()]

In [18]:
test.to_csv("data\\test.csv")

In [548]:
data_vivienda.to_csv("data\\vivienda.csv")

### FOURSQUARE API

In [21]:
client_id = open("..\\keys\\foursquare_key.txt").readlines()[0]
client_secret = open("..\\keys\\foursquare_key.txt").readlines()[1]
version = '20180323'

In [22]:
def extract(url):
    
    results = requests.get(url)

    code = json.loads(results.text)

    decoding = code.get("response")

    decoded = decoding.get("groups")[0]

    return decoded.get("items")

In [23]:
def getFromDict(diccionario,objeto):
    return reduce(operator.getitem,objeto,diccionario)

def places_df(frame,category):
    
    nombre = ["venue","name"]
    latitud = ["venue", "location","lat"]
    longitud = ["venue","location","lng"]
    barrio = ["venue", "location", "neighborhood"]
    tipo = ["venue", "categories", "pluralName"]
    x = []
    
    for diccionario in frame:
        lista = {}
        lista["name"] = getFromDict(diccionario,nombre)
        lista["longitude"] = getFromDict(diccionario,longitud)
        lista["latitude"] = getFromDict(diccionario,latitud)

        
        x.append(lista)
        
        frame = pd.DataFrame(x)

    c = [category for _ in range(len(frame))]
        
    frame["category"] = c
    
    return frame

In [350]:
location = "40.462781, -3.686737"
limit = 10
cat_id = "4bf58dd8d48988d116941735"
radius = 100
url=f'https://api.foursquare.com/v2/venues/explore?&client_id={client_id}&client_secret={client_secret}&v={version}&limit=200&ll={location}&radius={radius}&limit={limit}&categoryId={cat_id}'
s = extract(url)

In [24]:
def locations(radius, limit,location, cat_id, category ):
    url = f'https://api.foursquare.com/v2/venues/explore?&client_id={client_id}&client_secret={client_secret}&v={version}&limit=200&ll={location}&radius={radius}&limit={limit}&categoryId={cat_id}'
    x = extract(url)
    return places_df(x, category)

### Find establishments using Foursquare to populate database

In [25]:
# IDs
nightlife = "4d4b7105d754a06376d81259"
gym = "4bf58dd8d48988d175941735"
park = "4bf58dd8d48988d163941735"
hospital = "4bf58dd8d48988d196941735"
elementary_school = "4f4533804b9074f6e4fb0105"
high_school = "4bf58dd8d48988d13d941735"
middle_school = "4f4533814b9074f6e4fb0106"
clothing_store = "4bf58dd8d48988d103951735"
daycare = "4f4532974b9074f6e4fb0104"
grocery_store = "4bf58dd8d48988d118951735"
supermarket = "52f2ab2ebcbc57f1066b8b46"
pharmacy = "4bf58dd8d48988d10f951735"
general_entertainment = "4d4b7104d754a06370d81259"
bar = "4bf58dd8d48988d116941735"
metro_station = "4bf58dd8d48988d1fd931735"
train_station = "4bf58dd8d48988d129951735"
bus_stop = "52f2ab2ebcbc57f1066b8b4f"
elementary_school = "4f4533804b9074f6e4fb0105"
high_school = "4bf58dd8d48988d13d941735"
restaurant = "4d4b7105d754a06374d81259"

#### Use of queries_complete function 

In [35]:
def queries(loc):

    return locations(5000, 100, loc, restaurants, "r")

def queries_complete(lista, column):
    result = list(map(queries, lista))
    x = pd.DataFrame(result[0])
    y = pd.DataFrame(result[1])
    w = pd.DataFrame(result[2])

    return pd.concat([x,y,w]).drop_duplicates(subset=column, keep='first')
        

In [251]:
lista = [("40.458635, -3.676851"), ("40.467426, -3.674765"), ("40.460371, -3.685486")]

In [252]:
def queries(loc):

    return locations(1000, 100, loc, restaurant, "Restaurants and Nightlife")

restaurants = queries_complete(lista, "name")

In [253]:
def queries(loc):

    return locations(1000, 100, loc, hospital, "medical_centre")

hospitals = queries_complete(lista, "name")

In [254]:
def queries(loc):

    return locations(1000, 100, loc, nightlife, "Restaurants and Nightlife")

nightlifes = queries_complete(lista, "name")

In [255]:
restaurants_nightlife = pd.concat([restaurants, nightlifes])

In [256]:
def queries(loc):

    return locations(1000, 5, loc, gym, "gym")

gyms = queries_complete(lista, "latitude")

In [257]:
def queries(loc):

    return locations(1000, 100, loc, supermarket, "supermarket")
supermarkets = queries_complete(lista, "latitude")

In [258]:
def queries(loc):

    return locations(1000, 100, loc, grocery_store, "supermarket")

grocery = queries_complete(lista, "latitude")

In [259]:
supermarkets = pd.concat([supermarkets, grocery]).drop_duplicates(subset="latitude", keep='first')

In [260]:
def queries(loc):

    return locations(1000, 100, loc, general_entertainment, "general_entertainment")

ent = queries_complete(lista, "name")

In [261]:
def queries(loc):

    return locations(1000, 100, loc, pharmacy, "pharmacy")

pharmacies = queries_complete(lista, "latitude")

In [262]:
def queries(loc):

    return locations(1000, 100, loc, metro_station, "transport")

metro = queries_complete(lista, "name")

In [263]:
def queries(loc):

    return locations(1000, 100, loc, train_station, "transport")

train = queries_complete(lista, "latitude")

In [264]:
train = train[(train["name"].str.contains("Cercanías")) | train["name"].str.contains("Intercambiador") | train["name"].str.contains("Estación")]

In [265]:
train = train.drop(train[(train["name"].str.contains("Andén")) | train["name"].str.contains("Vía")| train["name"].str.contains("AVE") | train["name"].str.contains("VIP")].index)

In [266]:
transport = pd.concat([train, metro])

In [267]:
def queries(loc):

    return locations(5000, 100, loc, clothing_store, "general_entertainment")

store = queries_complete(lista, "latitude")

In [268]:
ent = pd.concat([store, ent])

In [269]:
def queries(loc):

    return locations(5000, 100, loc, park, "park")

parks = queries_complete(lista, "latitude")

In [270]:
parks = parks[(parks["name"].str.contains("Parque"))]

In [271]:
def queries(loc):

    return locations(5000, 100, loc, high_school, "school")

schools = queries_complete(lista, "latitude")

### Turn into geoframe and insert into MongoDB

#### Restaurants and Nightlife

#### Adding new data to mongo collection

In [272]:
total_1 = pd.concat([restaurants_nightlife, schools, hospitals, ent, pharmacies, gyms, parks, supermarkets,transport])

In [273]:
total_1 = geo_frame(total_1)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [274]:
conjunto_8 = pd.concat([total_1, conjunto_7])

In [275]:
conjunto_8.shape

(4344, 6)

In [288]:
conjunto_8.duplicated(subset="latitude").sum()

0

In [277]:
conjunto_8 = conjunto_8.drop_duplicates(subset="latitude", keep="first")

In [286]:
conjunto_8.shape

(3850, 5)

In [28]:
m = pd.read_csv("..\\data\\mongo.csv")

In [32]:
m.duplicated(subset="latitude").sum()

0

In [31]:
m = m.drop_duplicates(subset="latitude", keep='first')

In [284]:
conjunto_8 = conjunto_8.drop("_id", axis = 1)

In [285]:
conjunto_8.isnull().sum()

name         0
longitude    0
latitude     0
category     0
geometry     0
dtype: int64

In [287]:
insert_object(conjunto_8, p)

'Data succesfully added'

In [289]:
conjunto_8.to_csv("..\\data\\places.csv")